### Challenge 25: CTR bitflipping

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

There are people in the world that believe that CTR resists bit flipping attacks of the kind to which CBC mode is susceptible.

Re-implement the CBC bitflipping exercise from earlier to use CTR mode instead of CBC mode. Inject an "admin=true" token.

</div>

The CBC bit-flipping attack was [Challenge 16](16.ipynb).  

In [1]:
import cryptopals as cp
from Crypto import Random

<div class="alert alert-block alert-info">
Generate a random AES key.
</div>

I just discovered the Crypto random library -- which has get_random_bytes and getrandbits.  Will use those from here out.

In [2]:
# ---------UNKNOWN PARAMETERS ------------
unknown_key = Random.get_random_bytes(16)
unknown_nonce = Random.get_random_bytes(8)
# ---------END UNKNOWN PARAMETERS---------

Modified description from Exercise 16 to use CTR mode in place of CBC:

<div class="alert alert-block alert-info">
    
Combine your padding code and ~~CBC~~ **CTR** code to write two functions.

The first function should take an arbitrary input string, prepend the string:

```"comment1=cooking%20MCs;userdata="```

.. and append the string:

```";comment2=%20like%20a%20pound%20of%20bacon"```

The function should quote out the ";" and "=" characters.

The function should then pad out the input to the 16-byte AES block length and encrypt it under the random AES key.
</div>

In [3]:
def Challenge26Part1(data, key, nonce):

    prepend_str = b'comment1=cooking%20MCs;userdata='
    append_str = b';comment2=%20like%20a%20pound%20of%20bacon'

    data = data.replace(b';', b'\";\"')
    data = data.replace(b'=', b'\"=\"')

    plaintext = prepend_str + data + append_str
    plaintext = cp.PKCS7_pad(plaintext)

    ciphertext = cp.AESEncrypt(plaintext, key, 'CTR', nonce)

    return(ciphertext)

<div class="alert alert-block alert-info">
    
The second function should decrypt the string and look for the characters ";admin=true;" (or, equivalently, decrypt, split the string on ";", convert each resulting string into 2-tuples, and look for the "admin" tuple).

Return true or false based on whether the string exists.
</div>

In [4]:
def Challenge26Part2(ciphertext, key, nonce):

    plaintext = cp.AESDecrypt(ciphertext, key, 'CTR', nonce)
    plaintext = cp.strip_PKCS7_pad(plaintext)

    if plaintext.find(b';admin=true;') >= 0:
        return(True)
    else:
        return(False)

<div class="alert alert-block alert-info">
If you've written the first function properly, it should not be possible to provide user input to it that will generate the string the second function is looking for. We'll have to break the crypto to do that.
</div>

Let's try the trivial solution again -- just to confirm our functions work properly and reject it.

In [5]:
# Try the trivial solution:

payload = b';admin=true'
ct = Challenge26Part1(payload, unknown_key, unknown_nonce)

# Check to see if we were successful...

if Challenge26Part2(ct, unknown_key, unknown_nonce) is True:
    print('*** Success')
else:
    print('*** Try again loser!!!')

*** Try again loser!!!


---
Here was the hint for CBC mode.  How do we use what we learned there for CTR mode?

<div class="alert alert-block alert-info">
    
Instead, modify the ciphertext (without knowledge of the AES key) to accomplish this.

You're relying on the fact that in CBC mode, a 1-bit error in a ciphertext block:
- Completely scrambles the block the error occurs in
- Produces the identical 1-bit error(/edit) in the next ciphertext block.

    <div class="alert alert-block alert-warning">
        
    ### Stop and think for a second.
    
    Before you implement this attack, answer this question: why does CBC mode have this property?
        
    </div>
    
</div>

We should be able to do the same thing...except without corrupting any data this time because of how CTR mode works.  We just need to construct some carefully chosen user data and do a cut & paste.  

First things first -- we need to figure out what we are controlling with our user data...

In [6]:
payload = b''
ct = Challenge26Part1(payload, unknown_key, unknown_nonce)
payload2 = b'x'
ct2 = Challenge26Part1(payload2, unknown_key, unknown_nonce)

Find the index of the first byte that's different between the two ciphertexts -- this is where we have control.

In [7]:
idx = 0
while ct[idx] == ct2[idx]:
    idx+=1
print(f"First difference found at byte {idx}")

First difference found at byte 32


---
Now, supply user data to learn the key-stream for 11 bytes starting at that index: 

In [8]:
payload = b'\x00'*11
ct = Challenge26Part1(payload, unknown_key, unknown_nonce)
key_bytes = ct[idx:idx+11]

Now edit the ciphertext to what we want.

In [9]:
payload = b';admin=true'
edited_ct = ct[0:idx] + cp.bitwise_xor(payload, key_bytes) + ct[idx+11:]

And check to see if it worked...

In [10]:
if Challenge26Part2(edited_ct, unknown_key, unknown_nonce) is True:
    print('*** Success')
else:
    print('*** Try again loser!!!')

*** Success


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)